# Algoritmo SVM Clasificador

## Noelia Otazo Rojo

### Importamos las librerías

In [19]:
# Comando para la instalación de las librerías necesarias
#!pip install matplotlib keras tensorflow scikit-learn seaborn pandas plotly

import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
import numpy as np
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import classification_report

### Carga del dataset y transformación de datos

In [3]:
# Cargar el dataset de emergencias del 112 en CYL
data = pd.read_csv(".\dataset\Accidentes.csv", encoding="cp1252", sep=";")

# Antes de nada borramos los valores nulos
data_clean=data.dropna()

# Visualizar los primeros registros
# data.head()

# Separamos los datos en entrenamiento y prueba.
data_train, data_test = train_test_split(data_clean, test_size=0.3, stratify=data_clean['Tipo Accidente'])

#Transforma los datos a numéricos
#Tipo Accidente
map_target_class = {'Atropello a peatón': 0, 'Colisión lateral y frontolateral': 1, 'Colisión por alcance y múltiple': 2, 'Otro tipo de accidente': 3, 'Salida de la vía': 4, 'Colisión frontal': 5, 'Colisión lateral': 6, 'Vuelco en la calzada': 7}
data_train['TipoAccidente_Numérico'] = data_train['Tipo Accidente'].map(map_target_class)
data_test['TipoAccidente_Numérico'] = data_test['Tipo Accidente'].map(map_target_class)

#Sexo
map_target_sexo = {'Mujer': 0, 'Hombre': 1}
data_train['Sexo_Numérico'] = data_train['Sexo'].map(map_target_sexo)
data_test['Sexo_Numérico'] = data_test['Sexo'].map(map_target_sexo)

#Accesorios de seguridad
map_target_accesorios = {'NINGUNO': 0, 'SE DESCONOCE': 1, 'UTILIZANDO CINTURÓN ': 3, 'SISTEMA DE RETENCIÓN': 4, 'UTILIZANDO CASCO': 5, 'CON REFLECTANTES': 6}
data_train['Accesorios_Numérico'] = data_train['Accesorios de seguridad'].map(map_target_accesorios)
data_test['Accesorios_Numérico'] = data_test['Accesorios de seguridad'].map(map_target_accesorios)

#Tipo de vehículo
map_target_tipo_vehiculo = {'Ambulancia': 0, 'Autobús': 1, 'Bicicleta o triciclo sin motor': 2, 'Camión MMA <= 3500 Kg': 3, 'Camión MMA > 3500 Kg': 4, 'Ciclomotor': 5, 'Desconocido': 6, 'Furgoneta': 7, 'Motocicleta': 8, 'Otros Vehículos': 9, 'Otros vehículos ligeros': 10, 'Turismo': 11, 'Vehículo articulado': 12}
data_train['Vehículo_Numérico'] = data_train['Tipo de vehículo'].map(map_target_tipo_vehiculo)
data_test['Vehículo_Numérico'] = data_test['Tipo de vehículo'].map(map_target_tipo_vehiculo)


# data_train.head(2)

data_train.dropna()
data_test.dropna()

X_train = data_train.drop(['Tipo Accidente', 'Sexo', 'Accesorios de seguridad', 'Tipo de vehículo', 'TipoAccidente_Numérico'], axis=1)
y_train = data_train['TipoAccidente_Numérico']
X_test  = data_test.drop(['Tipo Accidente', 'Sexo', 'Accesorios de seguridad', 'Tipo de vehículo', 'TipoAccidente_Numérico'], axis=1)
y_test  = data_test['TipoAccidente_Numérico']

# Aplica dropna() mientras X_train y X_test son todavía DataFrames de pandas
X_train = X_train.dropna()
X_test = X_test.dropna()

# Ahora, alinea y_train y y_test con los índices de X_train y X_test después de dropna()
y_train = data_train['TipoAccidente_Numérico'].loc[X_train.index]
y_test = data_test['TipoAccidente_Numérico'].loc[X_test.index]

print(X_train.shape[0] == y_train.shape[0])  # Debería ser True
print(X_test.shape[0] == y_test.shape[0])  # Debería ser True

True
True


### Escalado de características

In [14]:
sc = StandardScaler()

X_train = sc.fit_transform(X_train)
X_test= sc.transform(X_test)

In [15]:
print(X_train.shape)
print(y_train.shape)

(2609, 9)
(2609,)


In [16]:
svc = SVC(probability=True)

svc.fit(X_train,y_train)

y_pred_svc = svc.predict(X_test)

In [17]:
print("Accuracy del modelo: ", svc.score(X_test,y_test))

Accuracy del modelo:  0.17271922054915856


In [22]:
print(classification_report(y_test, y_pred_svc))

              precision    recall  f1-score   support

           0       0.46      0.14      0.21       136
           1       0.16      0.53      0.25       192
           2       0.12      0.12      0.12       185
           3       0.15      0.07      0.10       171
           4       0.19      0.12      0.14       179
           5       0.67      0.03      0.06       138
           7       0.16      0.12      0.14       128

    accuracy                           0.17      1129
   macro avg       0.27      0.16      0.15      1129
weighted avg       0.26      0.17      0.15      1129



### Conclusión

Como podemos observar en la métrica anterior la puntuación del acierto de este modelo es de 0.17.  
**La precisión y el recall** para la mayoría de las clases son bajos, lo que significa que el clasificador tiene problemas para identificar correctamente las clases positivas (es decir, está clasificando incorrectamente los ejemplos de otras clases).   
**Los valores de F1-score** son también bajos, lo que sugiere que no hay un buen equilibrio entre la precisión y el recall.   
Parece que el **soporte** (el número de ocurrencias de cada clase en y_true) es desigual, lo que podría significar que tu conjunto de datos está desbalanceado. Esto puede afectar el rendimiento del clasificador,Lo que indica que puede estar sesgado hacia las clases más frecuentes. Algunas clases tienen una precisión relativamente más alta, como la clase 5 con una precisión del 67%, pero un recall muy bajo del 3%, lo que indica que mientras que las predicciones de esta clase suelen ser correctas, el modelo está fallando en identificar la mayoría de los casos positivos de esta clase.
